# Lab 4 - databehandling
I labøvelsen samlet du inn data for beregning av kalorimeterets varmekapasitet (Tabell 1 i labheftet bortsett fra kolonnen $C_\mathrm{cal}$). Lag en fil med disse dataene. Den kan for eksempel se ut noe sånt som
```
"t_K / degC", "t_V / degC", "t_mix / degC"
12.9, 42.3, 25.5
...
```
Les dataene inn i variabler. Hvis du er usikker på hvordan, kan du se på øvelse [2](../lab2/lab2.ipynb) og [3](../lab3/lab3.ipynb) der NumPy sin `genfromtxt` ble brukt.

Nå som dataene er lest inn i variabler, kan kalorimeterets varmekapasitet $C_\mathrm{cal}$ regnes ut ved
$$C_\mathrm{cal}=\left(\frac{t_\mathrm{V}-t_\mathrm{mix}}{t_\mathrm{mix}-t_\mathrm{K}}V_\mathrm{V}-V_\mathrm{K}\right)\rho_{H_2O}c_\mathrm{H_2O}.$$
Regn kalorimeterets gjennomsnittlige varmekapasitet i cellen under.

Lag en tilsvarende file for kolonnene $t_\mathrm{HNO_3}$, $t_\mathrm{NaOH}$ og $t_\mathrm{slutt}$ i Tabell 2. Les dataene inn i variabler og regn ut $t_\mathrm{start}$ ved
$$t_\mathrm{start} = \frac{V_\mathrm{HNO_3}t_\mathrm{HNO_3} + V_\mathrm{NaOH}t_\mathrm{NaOH}}{V_\mathrm{HNO_3} + V_\mathrm{NaOH}}$$
og temperaturforskjellen $\Delta t$ gitt ved
$$\Delta t = t_\mathrm{slutt} - t_\mathrm{start}.$$

# Statistikk

## Lineærregresjon med `polyfit`
I [tredje labøvelse](../lab3/lab3.ipynb) ble det gjort lineærregresjon med SciPy sin statistikkmodul. Men NumPy sin `polyfit` kan også brukes. Den tilpasser dataene til et polynom, og siden lineære funksjoner er førstegradspolynomer kan lineærregresjon utføres. Under er det gitt en måte å gjøre dette på. Ved å spesifisere `cov=True` får man også data om usikkerheten til regresjonskoeffisientene. `a` er stigningstallet og `b` er skjæringspunktet.

In [ ]:
(a, b), cov = np.polyfit(volume, temperature, 1, cov=True)
print('DT = %.3g*V%+.1g' % (a, b))

Nå kan dataene og regresjonlinjen plottes i det samme vinduet. Ser det ut som skjæringspunktet er omtrent 0?

## Students $t$-test
I prelabben ble det vist at temperaturøkningen er proporsjonal med volum tilsatt syre (forutsatt at all syren reagerer). Denne påstanden kan undersøkes med hypotesetest. 

### Hypotesene
Hvis måledataene våre er normalfordelte (ofte er de tilnærmet det), kan Students $t$-test brukes for å undersøke om en hypotese er statistisk signifikant. Her skal hypotsen
$$H_0: \text{Skjæringspunktet er null ($b = 0$)}$$
testes mot den alternative hypotesen
$$H_\mathrm{a}: \text{Skjæringspunktet er $ikke$ null ($b \neq 0$)}.$$

### Fun fact
*Student* i "Students $t$-test" har ikke direkte noe med studenter å gjøre. Det var pseudonymet til kjemikeren W. S. Gosset som jobbet ved Guinness' bryggeri i Dublin. Det sies at enten så foretrakk arbeidsgiveren hans at de ansatte publiserte under pseudonymer eller at de ikke fikk lov til å publisere. Uansett bruke han pseudonymet *Student* når han publiserte artiklene sin om $t$-testen, og den ble derfor gjent som Students $t$-test. Ved Guinness brukte Gosset $t$-testen til å undersøke kvaliteten til bygg og stout.

### $t^*$-observabelen
For å utføre $t$-testen trenger man en såkalt $t^*$-observabel. I [andre labøvelse](../lab2/lab2.ipynb) ble også $t$-verdier brukt, men var en signifikansverdie bestemt og det ble slått opp hvilken $t$-verdi dette tilsvarte. I dette tilfellet vil $t$-verdien bli regnet ut og slå opp hvilket signifikansverdi dette tilsvarer. $t^*$ regnes ut ved
$$t^*=\frac{b}{\mathrm{SE}_b}$$
der $b$ er skjæringspunktet og $\mathrm{SE}_b$ er standardfeilen til $b$. Under er det gitt hvordan $\mathrm{SE}_b$ regnes ut. Regn ut $t^*$.

In [ ]:
SE_b = math.sqrt(cov[1, 1])

### $P$-verdi
$P$-verdien er sannsynligheten for å observere en mer ekstrem verdi (i dette tilfellet større eller mindre) for $t^*$ gitt at nullhypotesen er sann.

Dersom $P$-verdien er liten, altså at vi vil sjeldent se mer ekstreme verdier enn den observerte, må nullhypotensen forkastes. Man må på forhånd bestemme liten $P$-verdien skal være og kaller denne verdien signifikansnivået $\alpha$ og settes ofte til $5 \%$ eller lavere. Dersom $P\leq\alpha$ sier man at $H_0$ forkastes med signifikans $\alpha$.  

Det er ingen [enkel formel](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Cumulative_distribution_function) for $P$-verdien, men SciPy har heldigvis noen innebyggede funksjoner til hjelp. Under er det gitt en funksjon for å regne ut $P$-verdien.

In [ ]:
P = t.sf(np.abs(t_stat), np.size(volume) - 2) * 2

Undersøk under om skjæringspunktet er signifikant lik null, bruk signifikansnivå $\alpha=5\%$.

### Reaksjonsvarme
Nå som du har stigningstallet kan reaksjonsvarmen regnes ut. Bruk formelen
$$\Delta t = -q_\mathrm{rxn}k\cdot V_\mathrm{a}$$
der
$$k=\frac{[\mathrm{HA}]_0}{\rho_\mathrm{H_2O}V_0c_\mathrm{H_2O}+C_\mathrm{cal}}$$
til å regne ut $q_\mathrm{rxn}$. Standardfeilen til $q_\mathrm{rxn}$ `SE_a` er gitt under. Oppgi reaksjonsvarmen som $q_\mathrm{rxn} \pm \mathrm{SE}$.

In [ ]:
SE_a = math.sqrt(cov[0, 0])